In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
import matplotlib.pyplot as plt

# Set device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Transformations: Resize and Normalize the images
transform = transforms.Compose([
    transforms.Resize((128, 128)),   # Resize images to a uniform size
    transforms.ToTensor(),           # Convert images to tensors
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])  # Normalize (adjust according to your data)
])

# Load the dataset
dataset = datasets.ImageFolder(root=r"C:\Users\losif\Desktop\FAI\summer 24\vision projects\data", transform=transform)

# Split dataset into training (80%), validation (10%), and testing (10%)
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

train_dataset, val_dataset, test_dataset = random_split(dataset, [train_size, val_size, test_size])

# Create DataLoader for each split
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# Check class-to-index mapping
print(dataset.class_to_idx)  # {'with_mask': 0, 'without_mask': 1}


{'with_mask': 0, 'without_mask': 1}


## Creating the CNN model

In [2]:
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(128 * 16 * 16, 512)  #! is this size right? i think yes...
        self.fc2 = nn.Linear(512, 2)  # 2 classes: with_mask and without_mask

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)  # flattended tensor
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize the model, define the loss function and optimizer
model = CNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)


In [3]:
def train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10):
    for epoch in range(epochs):
        model.train()  # Set the model to training mode
        running_loss = 0.0
        correct = 0
        total = 0

        # Training loop
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  # Zero the parameter gradients
            
            # Forward pass
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            
            # Backward pass and optimize
            loss.backward()
            optimizer.step()

            # Calculate accuracy
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
            running_loss += loss.item()

        train_loss = running_loss / len(train_loader)
        train_accuracy = 100 * correct / total

        # Validation loop
        model.eval()  # Set the model to evaluation mode
        val_loss = 0.0
        correct_val = 0
        total_val = 0

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                _, predicted = torch.max(outputs, 1)
                total_val += labels.size(0)
                correct_val += (predicted == labels).sum().item()

        val_loss /= len(val_loader)
        val_accuracy = 100 * correct_val / total_val

        print(f"Epoch [{epoch+1}/{epochs}], Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}%, "
              f"Val Loss: {val_loss:.4f}, Val Acc: {val_accuracy:.2f}%")

# Train the model
train_model(model, criterion, optimizer, train_loader, val_loader, epochs=10)


d:\Python\Lib\site-packages\PIL\Image.py:1056: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Epoch [1/10], Train Loss: 0.3860, Train Acc: 82.36%, Val Loss: 0.2390, Val Acc: 90.33%
Epoch [2/10], Train Loss: 0.2406, Train Acc: 89.61%, Val Loss: 0.2012, Val Acc: 91.66%
Epoch [3/10], Train Loss: 0.1851, Train Acc: 92.77%, Val Loss: 0.1997, Val Acc: 92.58%
Epoch [4/10], Train Loss: 0.1418, Train Acc: 94.69%, Val Loss: 0.1993, Val Acc: 92.72%
Epoch [5/10], Train Loss: 0.1129, Train Acc: 95.66%, Val Loss: 0.1613, Val Acc: 93.77%
Epoch [6/10], Train Loss: 0.0996, Train Acc: 96.54%, Val Loss: 0.1412, Val Acc: 95.76%
Epoch [7/10], Train Loss: 0.0775, Train Acc: 97.35%, Val Loss: 0.1724, Val Acc: 93.38%
Epoch [8/10], Train Loss: 0.0586, Train Acc: 97.83%, Val Loss: 0.2237, Val Acc: 94.44%
Epoch [9/10], Train Loss: 0.0316, Train Acc: 99.04%, Val Loss: 0.2039, Val Acc: 94.70%
Epoch [10/10], Train Loss: 0.0355, Train Acc: 98.73%, Val Loss: 0.2045, Val Acc: 94.70%


## **Evaluation time!** 

In [4]:
def test_model(model, test_loader):
    model.eval()  # Set the model to evaluation mode
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = 100 * correct / total
    print(f'Test Accuracy: {accuracy:.2f}%')

# Test the model
test_model(model, test_loader)


Test Accuracy: 94.71%
